## Import Dependencies and Open Instrument Handles

In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")

from Driver_OSA_MS9740B import *
from Driver_TSL_Goumax import *

In [42]:
rm = pyvisa.ResourceManager()
osa = OSA_MS9740B()
osa.open(rm)
tsl = TSL_TLS200()
tsl.open(rm)

OSA Attenuation Status:  ON



## Initialization

In [37]:
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000

wavelength_start = 1550
wavelength_stop = 1600
wavelength_step = 5
wavelength = np.linspace(wavelength_start,wavelength_stop,int((wavelength_stop-wavelength_start)/wavelength_step)+1)
tsl.set_output_on()
tsl.set_wavelength(wavelength_start)

## Define Functions

In [38]:
def sweepOSA(osa, wavelength_target, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_target - 50, wavelength_target + 50, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_target - 50, wavelength_target + 50, wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

## Measurement Routine

In [39]:
meas_df = pd.DataFrame()
SOA_bias_mA = 550
for i in range(0,len(wavelength)):
    tsl.set_wavelength(wavelength[i])
    time.sleep(0.1)
    Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA(osa, wavelength[i], wavelength_points, OSA_Resolution, OSA_VBW)
    meas_df_temp = pd.DataFrame({'TSL_wavelength_target': wavelength[i],'OSA_wavelength_nm':Wavelength_list, 'OSA_power_dBm':Power_list, 'OSA_power_uW':Power_list_uW, 'OSA_wavelength_peak_nm':Wavelength_peak_nm, 'OSA_power_peak_nm':Power_peak_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                                '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
    data = [meas_df_temp, meas_df]
    meas_df = pd.concat(data,ignore_index=True,sort=False)

In [40]:
meas_df.to_csv(r"TSL_SOA_VOA_500mA_25C_R2.csv")

In [44]:
tsl.close()
osa.close()

In [33]:
meas_df

,TSL_wavelength_target,OSA_wavelength_nm,OSA_power_dBm,OSA_power_uW,OSA_wavelength_peak_nm,OSA_power_peak_nm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,1553.0,1548.000,-64.62,0.000345,1553.1,15.68,57.00,48.05,0.143,0.052
1,1553.0,1548.005,-58.97,0.001268,1553.1,15.68,57.00,48.05,0.143,0.052
2,1553.0,1548.010,-54.3,0.003715,1553.1,15.68,57.00,48.05,0.143,0.052
3,1553.0,1548.015,-54.05,0.003936,1553.1,15.68,57.00,48.05,0.143,0.052
4,1553.0,1548.020,-54.41,0.003622,1553.1,15.68,57.00,48.05,0.143,0.052
...,...,...,...,...,...,...,...,...,...,...
7999,1550.0,1554.980,-41.65,0.068391,1550.1,16.68,58.05,47.65,0.135,0.135
8000,1550.0,1554.985,-41.6,0.069183,1550.1,16.68,58.05,47.65,0.135,0.135
8001,1550.0,1554.990,-41.57,0.069663,1550.1,16.68,58.05,47.65,0.135,0.135
8002,1550.0,1554.995,-41.45,0.071614,1550.1,16.68,58.05,47.65,0.135,0.135


In [10]:
tsl.set_wavelength(1550)

In [43]:
tsl.set_output_off()

In [19]:
osa.osa.write(f"CENTER {wavelength[i]}")

15